**Author:** A.S. Grm (aleksander.grm@fpp.uni-lj.si)

**Date:** 2024

<hr>

# Position Calculation Method

This is a method for calculating the astronomical position when the observer's position is known. It can involve multiple celestial bodies!

Instead of plotting the position using a line of position (LOP), we can compute it using the method described in the **Admiralty Almanac**.

Given our assumed position **AP** ($\varphi_\text{AP},\lambda_\text{AP}$) and the height difference $\Delta h_i$ and azimuth $\omega_i$ for each celestial body, we can determine the calculated position using the following approach.

First, calculate the coefficients $A_1,B_1,C_1,D_1,E_1,G_1$ using the formulas:

$$ A_1 = \sum_{i=1}^n \cos^2 \omega_i, ~~~ B_1 = \sum_{i=1}^n \sin \omega_i \: \cos \omega_i, ~~~ C_1 = \sum_{i=1}^n \sin^2 \omega_i, ~~~ D_1 = \sum_{i=1}^n \Delta h_i \: \cos \omega_i, ~~~ E_1 = \sum_{i=1}^n \Delta h_i \: \sin \omega_i, ~~~ G_1 = A_1 \cdot C_1 - B_1^2,$$

where $i$ is the number of measured celestial bodies. Once the coefficients are computed, we can determine the observer's position:

$$ \varphi_1 = \varphi_\text{AP} + \frac{C_1 \cdot D_1 - B_1 \cdot E_1}{G_1}, ~~~~ \lambda_1 = \lambda_\text{AP} + \frac{A_1 \cdot E_1 - B_1 \cdot D_1}{G_1 \cdot \cos \varphi_\text{AP}}.$$

Now, with the new position ($\varphi_1, \lambda_1$), we can iterate to recalculate $\Delta h_i$ and $\omega_i$. We repeat the entire process to determine coefficients $A_2,B_2,C_2,D_2,E_2,G_2$ and compute a second new position:

$$ \varphi_2 = \varphi_1 + \frac{C_2 \cdot D_2 - B_2 \cdot E_2}{G_2}, ~~~~ \lambda_2 = \lambda_1 + \frac{A_2 \cdot E_2 - B_2 \cdot D_2}{G_2 \cdot \cos \varphi_1}.$$

This iteration continues until the difference between ($\varphi_k, \lambda_k$) and ($\varphi_{k+1}, \lambda_{k+1}$) is arbitrarily small.

The iteration in calculation is introduced because the line of position (LOP) is not a straight line but a circle. The closer the calculated position is to the true position, the smaller the error of this method, which is based on the assumption that the LOP is a straight line.

<hr>

In [ ]:
import os, sys
import subprocess as sp

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [ ]:
def get_coefficients(dh,w,nn):

    A = 0
    for i in range(nn):
        A += mat.cos(nt.deg2rad(w[i]))**2

    B = 0
    for i in range(nn):
        B += mat.sin(nt.deg2rad(w[i]))*mat.cos(nt.deg2rad(w[i]))

    C = 0
    for i in range(nn):
        C += mat.sin(nt.deg2rad(w[i]))**2

    D = 0
    for i in range(nn):
        D += dh[i]*mat.cos(nt.deg2rad(w[i]))

    E = 0
    for i in range(nn):
        E += dh[i]*mat.sin(nt.deg2rad(w[i]))

    G = A*C - B**2

    return [A,B,C,D,E,G]

In [ ]:
def get_dh_array(Ho, Hc, nn, convert=False):

    dhA = []
    for i in range(nn):
        if convert:
            dh = nt.dms2dd(Ho[i]) - nt.dms2dd(Hc[i]) 
        else:
            dh = Ho[i] - Hc[i]
        dhA.append(dh)

    return np.array(dhA)

In [ ]:
def get_position(P,dh,w,nn):

    [A,B,C,D,E,G] = get_coefficients(dh,w,nn)
    
    fi = P[0] + (C*D - B*E)/G
    la = P[1] + (A*E - B*D)/(G * mat.cos(nt.deg2rad(P[0])))

    return [fi,la]

In [ ]:
# Main program

# Date: 2021/05/11, UT: 19:00:00
# Bodies: Alioth, Regulus in Elnath
# Assumed position: 41 N, 20 W
# True position: 41 30'N, 20 30'W (for control)

fi = [29,48,'S']
la = [109,3,'W']
P = [nt.nav2dd(fi), nt.nav2dd(la)]   # assumed position
cbn = ['Denebola','Menkent','Miaplacidus','Rigel']  # celestial body names
Ho = [[27,9.5],[23,41.8],[49,28.2],[36,50.1]] # observed altitudes [deg, min]
Hc = [[27,16.9],[24,11.1],[49,47.7],[36,23.3]]   # calculated altitudes [deg, min]
w = [51.8, 119.6, 176.3,282.6]             # calculated azimuth
nn = 4                               # number of celestial bodies

dh = get_dh_array(Ho,Hc,nn,True)     # find the intercept
print('intercept:')
for i in range(nn):
    print(' -> {:s}: {:.1f} min'.format(cbn[i],dh[i]*60))

[fi, la] = get_position(P,dh,w,nn)   # new calculated astro fix

print()
print('  AP: fi={:s}, la={:s}'.format(nt.prettyPrintLat(P[0]),nt.prettyPrintLong(P[1])))
print('    ------------------------------')
print(' Fix: fi={:s}, la={:s} (calculated)'.format(nt.prettyPrintLat(fi),nt.prettyPrintLong(la)))
print('True: fi={:s}, la={:s} (for control)'.format(nt.prettyPrintLat(-29.5),nt.prettyPrintLong(-109.5)))